In [2]:
!pip install kaggle


In [3]:
import os
import json
from zipfile import ZipFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
kaggle_dict = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dict['username']
os.environ['KAGGLE_KEY'] = kaggle_dict['key']

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zipObj:
   zipObj.extractall()

In [7]:
df=pd.read_csv('IMDB Dataset.csv')

In [8]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [10]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [11]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})


<ipython-input-11-b67887bc710b>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})


In [12]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [13]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(df,test_size=0.2,random_state=42)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

In [16]:
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [17]:
Y_train=train_data['sentiment']
Y_test=test_data['sentiment']

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the model
model = Sequential()

# Add an Embedding layer (vocab_size=5000, output_dim=128)
model.add(Embedding(input_dim=5000, output_dim=128))

# Add an LSTM layer with dropout and recurrent dropout
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

# Add a Dense output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Check the model summary
model.summary()


In [24]:
from tensorflow.keras.callbacks import EarlyStopping

# Use EarlyStopping to stop training when validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train,Y_train,validation_split=0.2,epochs=5,batch_size=64, callbacks=[early_stopping])


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 231s 461ms/step - accuracy: 0.9411 - loss: 0.1605 - val_accuracy: 0.8736 - val_loss: 0.3412
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 259s 457ms/step - accuracy: 0.9500 - loss: 0.1381 - val_accuracy: 0.8535 - val_loss: 0.3765
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 229s 458ms/step - accuracy: 0.9513 - loss: 0.1281 - val_accuracy: 0.8761 - val_loss: 0.3773
